# 런타임 시작 시 먼저 실행

In [ ]:
from google.colab import drive
import sys

# 작업 경로
work_path = "/content/drive/MyDrive/ESTCampAI/시계열프로젝트"

# 구글 드라이브 연동
drive.mount("/content/drive")
%cd $work_path

# 라이브러리 경로 추가
sys.path.append(work_path)

# 라이브러리 임포트
from pandas import read_csv
from TextStyleTransfer import TrainProcess
from TextStyleTransfer import ModelPipeline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ESTCampAI/시계열프로젝트


# 사용자 입력 변수

In [ ]:
# 사용자 입력 변수
dataset_path = "./data/smilestyle_dataset.tsv"
model_path = "./data/TextStyleTransferTest/"
source_text = """
어쩌다 마주친 그대 모습에
내 마음을 빼앗겨 버렸네
어쩌다 마주친 그대 두 눈이
내 마음을 사로잡아 버렸네
그대에게 할 말이 있는데
왜 이리 용기가 없을까
음 말을 하고 싶지만 자신이 없어
내 가슴만 두근두근
답답한 이 내 마음
바람 속에 날려 보내리
"""

# 학습

In [ ]:
# 모델 학습
model = TrainProcess(
    model_name = "gogamza/kobart-base-v2"
  ).SplitTrainTestData(
      dataset_path = dataset_path,
      nan_allow = 2,
      test_size = 0.1,
      random_state = 1945
  ).SetDatasets(
      max_length = 1024,
      truncation = True
  ).SetTrainingArguments(
      output_dir = f"{model_path}dummy/",
      overwrite_output_dir = True,
      num_train_epochs = 16,
      per_device_train_batch_size = 16,
      per_device_eval_batch_size = 16,
      eval_steps = 500,
      warmup_steps = 300,
      prediction_loss_only = True,
      evaluation_strategy = "steps",
      save_strategy = "no",
      save_steps = 0,
      save_total_limit = 0,
      logging_dir = None,
      logging_strategy = "no",
      do_eval = False,
      do_predict = False,
      load_best_model_at_end = False,
      metric_for_best_model = 'eval_loss'
  ).Train(
  ).SaveModel(
      path = model_path
  )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


학습 데이터: (3123, 17), 테스트 데이터: (347, 17)

학습/테스트 데이터셋 생성.

Training Arguments 설정 완료.
 모델 학습 시 저장 경로:  ./data/TextStyleTransferTest/dummy/
Trainer 설정 완료



/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: good_neighbour (good_neighbour-sejong-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/content/drive/MyDrive/ESTCampAI/시계열프로젝트/TextStyleTransfer.py:352: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  encoder_text = f"{self.__StyleMap[row.index[1]]} 말투로 변환:{row[0]}"
/content/drive/MyDrive/ESTCampAI/시계열프로젝트/TextStyleTransfer.py:355: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  decoder_text = f"{row[1]}{self.__tokenizer.eos_token}"
/usr/local/lib/python3.12/dist-packages/transformers/to

Step,Training Loss,Validation Loss
500,No log,1.825647


KeyboardInterrupt: 

# 텍스트 스타일 변환 예시

In [ ]:
# 컬럼 이름 가져오기
columns = read_csv(dataset_path, sep = "\t").columns
print(columns)

# 스타일 변환 도구 생성
textGenerator = ModelPipeline(model_path = model_path)

# 원본 문장 출력
print("입력 문장:", source_text)

# 텍스트 생성
for style in columns:
  print(style, textGenerator.GenerateText(source_text, style, 1, 2048)[0])